# Web Proxy

In [2]:
import os 
import pandas as pd
from socket import *
serverSock = socket(AF_INET, SOCK_STREAM)
serverSock.bind(('', 5000))
serverSock.listen(1)
connectionSock, addr = serverSock.accept()
print(str(addr),'에서 접속이 확인되었습니다.')
data = connectionSock.recv(1024)
print('받은 데이터 : ', data.decode('utf-8'))
serverSock.close()

('127.0.0.1', 57421) 에서 접속이 확인되었습니다.
받은 데이터 :  GET http://127.0.0.1/vulnerabilities/sqli/?id=1&Submit=Submit HTTP/1.1
Host: 127.0.0.1
User-Agent: Mozilla/5.0 (X11; Linux i686; rv:24.0) Gecko/20140723 Firefox/24.0 Iceweasel/24.7.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Cookie: dtck_first=20191215; PHPSESSID=21004lnnv3op2mq4csvdcduie7; security=low
Connection: keep-alive




In [9]:
import os 
import pandas as pd
from socket import *

serverSock = socket(AF_INET, SOCK_STREAM)
clientSock = socket(AF_INET, SOCK_STREAM)
serverSock.bind(('', 5000))
serverSock.listen(1)
connectionSock, addr = serverSock.accept()
print(str(addr),'에서 접속이 확인되었습니다.')
data = connectionSock.recv(1024)
print('웹브라우저에서 받은 데이터 : ', data.decode('utf-8'))
clientSock.connect(('169.254.235.126', 80))
print('연결 확인 됐습니다.')
clientSock.send(data)
print('웹서버로 메시지를 전송했습니다.')
data = clientSock.recv(2048)
print('받은 데이터 : ', data)
clientSock.close()
serverSock.close()



('127.0.0.1', 57825) 에서 접속이 확인되었습니다.
웹브라우저에서 받은 데이터 :  GET http://localhost/DVWA HTTP/1.1
Host: localhost
User-Agent: Mozilla/5.0 (X11; Linux i686; rv:24.0) Gecko/20140723 Firefox/24.0 Iceweasel/24.7.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Connection: keep-alive


연결 확인 됐습니다.
웹서버로 메시지를 전송했습니다.
받은 데이터 :  b'HTTP/1.1 404 Not Found\r\nDate: Sun, 09 Aug 2020 01:28:21 GMT\r\nServer: Apache/2.2.22 (Debian)\r\nVary: Accept-Encoding\r\nContent-Encoding: gzip\r\nContent-Length: 231\r\nKeep-Alive: timeout=5, max=100\r\nConnection: Keep-Alive\r\nContent-Type: text/html; charset=iso-8859-1\r\n\r\n\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x03M\x8fAK\xc3@\x10\x85\xef\xf9\x15\xcf\x9e\xf4`&\x86\x1e<,\x0b\xb5I\xb1\x10k\xd0\xad\xe2q\xdb\x1d\xd9@\xcc\xc6\xdd\xad\xe2\xbfw\x93"\xc8\xc0\xc0\x9b\xf9\xde\xf0F\\T\x8fk\xf5\xd6\xd6\xb8W\x0f\r\xda\xfd]\xb3]cqM\xb4\xad\xd5\x86\xa8R\xd5yS\xe6\x05Q\xbd[\xc8L\xd8\xf8\xd1KaY\x9b

In [15]:
import pefile
import struct
import datetime
pe = pefile.PE("c:/imsi/USBDeview.exe")
print(pe.NT_HEADERS.Signature)
nt_header_list = []
nt_header_list.append(["Signature", struct.pack('<I', pe.NT_HEADERS.Signature).decode('utf8'), "NF Signature"])
print (nt_header_list)

17744
[['Signature', 'PE\x00\x00', 'NF Signature']]


In [16]:
import pefile
import struct
import datetime

def print_info (data_list) :
    for data in data_list :
        print(data[0].ljust(20), str(data[1]).ljust(20), data[2].ljust(20))


def dos_header_info (pe) :
    print("-" * 30)
    print("[DOS Header]에서 필요한 정보\n")
    dos_header_list = []
    dos_header_list.append(["실제 변수명", "값", "의미"])
    dos_header_list.append(["e_magic", struct.pack('<H', pe.DOS_HEADER.e_magic).decode('utf8'), "DOS Signature"])
    dos_header_list.append(["e_lfanew",hex(pe.DOS_HEADER.e_lfanew), "NT header offset"])
    
    print_info(dos_header_list)
    
    print("-" * 30,"\n")
    
    
    
def nt_header_info (pe) :
    print("-" * 30)
    print("[NT header]에서 필요한 정보\n")
    nt_header_list = []
    nt_header_list.append(["실제 변수명", "값", "의미"])
    nt_header_list.append(["Signature", struct.pack('<I', pe.NT_HEADERS.Signature).decode('utf8'), "NF Signature"])
    nt_header_list.append(["Machine", hex(pe.FILE_HEADER.Machine), "CPU 별 고유값 (x86 = 0x14c / x64 = 0x8664)"])
    
    timeStr = '1970-01-01 00:00:00'
    Thistime = datetime.datetime.strptime(timeStr, '%Y-%m-%d %H:%M:%S') 
    LastBuildtime = Thistime + datetime.timedelta(seconds=pe.FILE_HEADER.TimeDateStamp)

    nt_header_list.append(["TimeDateStamp", str(LastBuildtime), "파일을 빌드한 시간"])
    nt_header_list.append(["NumberOfSections", pe.FILE_HEADER.NumberOfSections, "Section의 총 개수"])
    nt_header_list.append(["SizeOfOptionalHeader", hex(pe.FILE_HEADER.SizeOfOptionalHeader), "OptionalHeader의 크기"])
    nt_header_list.append(["Characteristics", hex(pe.FILE_HEADER.Characteristics), "이 파일의 속성"])
    nt_header_list.append(["Magic", hex(pe.OPTIONAL_HEADER.Magic), "Optional header를 구분하는 Signature (32bit=10b / 64bit=20b)"])
    nt_header_list.append(["SizeOfCode", hex(pe.OPTIONAL_HEADER.SizeOfCode), "IMAGE_SCN_CNT_CODE 속성을 갖는 섹션들의 총 사이즈 크기"])
    nt_header_list.append(["AddressOfEntryPoint", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint), "PE 파일이 메모리 로드 후 처음 실행되어야 하는 코드 주소"])
    nt_header_list.append(["ImageBase",  hex(pe.OPTIONAL_HEADER.ImageBase), "PE파일이 매핑되는 시작주소"])
    nt_header_list.append(["SectionAlignment", pe.OPTIONAL_HEADER.SectionAlignment, "메모리 상에서의 최소 섹션 단위"])
    nt_header_list.append(["FileAlignment", pe.OPTIONAL_HEADER.FileAlignment, "파일 상에서의 최소 섹션 단위"])
    
    print_info (nt_header_list)
    
    print("-" * 30,"\n")   
   
    
def sections_header_info (pe) :
    print("-" * 30)
    print("[sections_info]에서 필요한 정보\n")
    print("\t개념")
    print("Name".ljust(20), "Section 이름을 나타냄")
    print("VirtualAddress".ljust(20), "섹션의 RAV(ImageBase + VA)를 위한 VA 값")
    print("SizeOfRawData".ljust(20), "파일 상에서 섹션이 차지하는 크기")
    print("PointerToRawData".ljust(20), "파일 상에서 섹션이 시작하는 위치")
    print("Characteristics".ljust(20), "섹션의 특징을 나타냄")
    print("".ljust(20), "(0x20000000 = excutable, 0x40000000 = readable, 0x80000000 = writeable, 0x00000020 = contains code, 0x00000040 = contains initialized data)")
    print("")
    
    print("Name".ljust(20), "Virtual Address".ljust(20), "SizeOfRawData".ljust(20),
          "PointerToRawData".ljust(20), "Characteristics".ljust(20))
    for section in pe.sections :     
         print(section.Name.decode('utf8').ljust(20), hex(section.VirtualAddress).ljust(20),
               hex(section.SizeOfRawData).ljust(20), hex(section.PointerToRawData).ljust(20), hex(section.Characteristics))      
         
    print("-" * 30,"\n")
    
    
pe = pefile.PE("c:/imsi/USBDeview.exe")
print("파싱 시작 ...")
dos_header_info(pe)
nt_header_info(pe)
sections_header_info(pe)

파싱 시작 ...
------------------------------
[DOS Header]에서 필요한 정보

실제 변수명               값                    의미                  
e_magic              MZ                   DOS Signature       
e_lfanew             0xf0                 NT header offset    
------------------------------ 

------------------------------
[NT header]에서 필요한 정보

실제 변수명               값                    의미                  
Signature            PE                   NF Signature        
Machine              0x14c                CPU 별 고유값 (x86 = 0x14c / x64 = 0x8664)
TimeDateStamp        2019-03-18 16:55:18  파일을 빌드한 시간          
NumberOfSections     4                    Section의 총 개수       
SizeOfOptionalHeader 0xe0                 OptionalHeader의 크기  
Characteristics      0x103                이 파일의 속성            
Magic                0x10b                Optional header를 구분하는 Signature (32bit=10b / 64bit=20b)
SizeOfCode           0x12e00              IMAGE_SCN_CNT_CODE 속성을 갖는 섹션들의 총 사이즈 크기
AddressOfEntryPoint  0

In [3]:
import pefile

pe = pefile.PE("C:/Lim/표준협회/챌린지파일/ec1093137cdb4f6482cf737631dfa301/train_set/000b53e07befbe136b3593641797bf982a399446480821384ff0f9a2107b1693.vir")
#pe = pefile.PE("C:/LimBest/표준협회/챌린지파일/0309fe817f06390d426ca0491783106e03d9886708bab28b60283ebeb2382bdf.vir")
#pe = pefile.PE("C:/LimBest/표준협회/챌린지파일/6f252415c510e8a69444e6135ecead02c74a294da4b2efee556a1fd2d9f407fc.vir")
#pe = pefile.PE("C:/temp/test.exe")

pe.parse_data_directories()
try:
   for entry in pe.DIRECTORY_ENTRY_IMPORT:
      for imp in entry.imports:
         try :
            
            print('\t', hex(imp.address), imp.name)
         except:
            pass
except:
   pass


	 0x41e660 b'LoadLibraryA'
	 0x41e664 b'GetProcAddress'
	 0x41e668 b'VirtualAlloc'
	 0x41e66c b'VirtualFree'


# 하나의 파일 압축풀기

In [21]:
import zipfile

path_to_zip_file = 'C:/temp/test.docm'
directory_to_extract_to = 'C:/temp/result'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)


# 디렉터리 전체를 압축하기

In [24]:
import os
import zipfile
new_zips= zipfile.ZipFile('c:/temp/test.zip', 'w')
for folder, subfolders, files in os.walk('c:/temp/result'):
    for file in files:
        new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), 'c:/temp/'), compress_type = zipfile.ZIP_DEFLATED)
new_zips.close()


c:/temp/result result.docm
c:/temp/result [Content_Types].xml
c:/temp/result ~$result.docm
c:/temp/result\docProps app.xml
c:/temp/result\docProps core.xml
c:/temp/result\word document.xml
c:/temp/result\word fontTable.xml
c:/temp/result\word settings.xml
c:/temp/result\word styles.xml
c:/temp/result\word webSettings.xml
c:/temp/result\word\theme theme1.xml
c:/temp/result\word\_rels document.xml.rels
c:/temp/result\word\_rels vbaProject.bin.rels
c:/temp/result\_rels .rels


# 파일삭제

In [28]:
import os 
# 디렉터리 삭제 
# os.rmdir("c:/temp/test")

#os.remove("c:/temp/test.zip")

# 특정 디렉터리 및 파일삭제 
# 서브 디렉터리까지 모두 삭제 
import shutil 
shutil.rmtree("c:/temp/abc")


In [35]:
test = "ABD \ TEST AutoOpen DDDD ZZZ"
index=test.find("AutoOpen")
print(test[index])

A


In [53]:
import os
 
def search(dir):
        files = os.listdir(dir)
        for file in files:
                print(file)
 
search("c:/temp/result/word")

document.xml
fontTable.xml
settings.xml
styles.xml
theme
vbaData.xml
vbaProject.bin
webSettings.xml
_rels


In [69]:
import os
 
def search(dir):
        files = os.listdir(dir)
        for file in files:
                fullFilename = os.path.join(dir, file)
                if os.path.isdir(fullFilename):
                        search(fullFilename)
                else:
                        print(fullFilename)
 
search("/temp")

/temp\test.docm
/temp\test.txt
/temp\test2.docx
/temp\오목눈이\test.docm


In [68]:
import os
import sys
 
def search1(dir):
        files = os.listdir(dir)
        for file in files:
                fullFilename = os.path.join(dir, file)
                if os.path.isdir(fullFilename):
                        search(fullFilename)
                else:
                        print(fullFilename)
 
searchDir = "c:/temp/"
search1(searchDir)
 

c:/temp/test.docm
c:/temp/test.txt
c:/temp/test2.docx


# CDR: MSWord 매크로 탐지기능

In [2]:
import zipfile
import os
import sys


# 특정 파일 압축을 해제한다.
def UnzipFile(path_to_zip_file, directory_to_extract_to) :
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)

def ZipFile(path_to_zip_file, directory_to_extract_to) :
    new_zips= zipfile.ZipFile(path_to_zip_file, 'w')
    for folder, subfolders, files in os.walk(directory_to_extract_to):
        for file in files:
            new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), resultdir), compress_type = zipfile.ZIP_DEFLATED)
    new_zips.close()


def search(dir):
    files = os.listdir(dir)
    for file in files:
        if file == 'vbaProject.bin' :
            return True
    return False

def MacroFile(filepath) :
    f = open(filepath, "r")
    while True:
        data = f.readline()
        if not data: break
        position=data.find("AutoOpen")
        if position >= 1 :
            f.flush()
            f.close()
            return True
    f.flush()
    f.close()
    return False
    
    
orgfile = "C:/temp/문서1.docm"
resultdir="c:/temp/result"
resultpath="c:/temp"

# UnzipFile(orgfile , resultdir)
macrodir = resultdir+"/word"
detection = search(macrodir)

if detection == True :
    filepath= macrodir + "/vbaData.xml"
    autocheck = MacroFile(filepath)
    if autocheck == True :
        print("매크로 자동실행 기능이 포함되어 있습니다.")
        ZipFile("c:/temp/reuslt.zip", resultdir)




매크로 자동실행 기능이 포함되어 있습니다.


In [1]:
import zipfile
import os
import sys

orgfile = "C:/temp/문서1.docm"
resultdir="c:/temp/result"
resultpath="c:/temp"

def UnzipFile(path_to_zip_file, directory_to_extract_to) :
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)

UnzipFile(orgfile , resultdir)

In [ ]:
import zipfile
import os
import sys


# 특정 파일 압축을 해제한다.
def UnzipFile(path_to_zip_file, directory_to_extract_to) :
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)

def ZipFile(path_to_zip_file, directory_to_extract_to) :
    new_zips= zipfile.ZipFile(path_to_zip_file, 'w')
    for folder, subfolders, files in os.walk(directory_to_extract_to):
        for file in files:
            new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), resultdir), compress_type = zipfile.ZIP_DEFLATED)
    new_zips.close()


def search(dir):
    files = os.listdir(dir)
    for file in files:
        if file == 'vbaProject.bin' :
            return True
    return False

def MacroFile(filepath) :
    f = open(filepath, "r")
    while True:
        data = f.readline()
        if not data: break
        position=data.find("AutoOpen")
        if position >= 1 :
            f.flush()
            f.close()
            return True
    f.flush()
    f.close()
    return False

orgfile = "C:/temp/test.docm"
resultdir="c:/temp/result"
resultpath="c:/temp"

while True:
    print("CDR 분석시스템 : ")
    print("1. 매크로 파일을 탐지")
    print("2. 매크로를 파일에서 제거")
    print("3. 종료")
    command = int(input())
    if command == 1 :
        print("매크로 탐지 파일명을 입력하세요")
        orgfile = str(input())
        UnzipFile(orgfile , resultdir)
        macrodir = resultdir+"/word"
        detection = search(macrodir)
        if detection == True :
            print("매크로를 포함고 있는 문서입니다.")
            filepath= macrodir + "/vbaData.xml"
            autocheck = MacroFile(filepath)
            if autocheck == True :
                print("자동 실행 기능이 포함되어 있습니다.")
        else :
            print("매크로가 없습니다.")
    if command == 2 :
        print("매크로를 강제적으로 삭제합니다.")
        orgfile = str(input())
        UnzipFile(orgfile , resultdir)
        filepath= macrodir + "/vbaData.xml"
        os.remove(filepath)
        os.remove(macrodir +'/vbaProject.bin')
        ZipFile("c:/temp/reuslt.zip", resultdir)
    if command == 3 :
        print("프로그램을 종료합니다")
        break
    


CDR 분석시스템 : 
1. 매크로 파일을 탐지
2. 매크로를 파일에서 제거
3. 종료


# 내 컴퓨터 있는 모든 매크로 파일을 찾아라

In [1]:
import os
import sys
import zipfile
import shutil 

def search(dir):
    files = os.listdir(dir)
    for file in files:
        if file == 'vbaProject.bin' :
            return True
    return False

def UnzipFile(path_to_zip_file, directory_to_extract_to) :
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)

def searchMacro(dir):
        files = os.listdir(dir)
        for file in files:
                fullFilename = os.path.join(dir, file)
                if os.path.isdir(fullFilename):
                        searchMacro(fullFilename)
                else:
                        filename  = fullFilename
                        wordcount = filename.count("\\")
                        filename = filename.replace("\\","/",wordcount)
                        data=filename.split('.')
                        if data[1]=="docm" or data[1]=="docx" or data[1]=="doc" :
                            print(filename)
                            UnzipFile(filename, resultdir)
                            macrodir = resultdir+"/word"
                            detection = search(macrodir)
                            if detection == True :
                                print(filename, "은 매크로를 포함하고 있습니다.")
                            shutil.rmtree(resultdir)                        
searchDir = "c:/temp"
resultdir="c:/temp/result"
searchMacro(searchDir)
 

In [75]:
import os
import sys
import zipfile
import shutil 
import cx_Oracle
import pandas as pd

def search(dir):
    files = os.listdir(dir)
    for file in files:
        if file == 'vbaProject.bin' :
            return True
    return False

def UnzipFile(path_to_zip_file, directory_to_extract_to) :
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)

def searchMacro(dir):
        files = os.listdir(dir)
        for file in files:
                fullFilename = os.path.join(dir, file)
                if os.path.isdir(fullFilename):
                        searchMacro(fullFilename)
                else:
                        filename  = fullFilename
                        wordcount = filename.count("\\")
                        filename = filename.replace("\\","/",wordcount)
                        data=filename.split('.')
                        if data[1]=="docm" or data[1]=="docx" or data[1]=="doc" :
                            print(filename)
                            UnzipFile(filename, resultdir)
                            macrodir = resultdir+"/word"
                            detection = search(macrodir)
                            if detection == True :
                                print(filename, "은 매크로를 포함하고 있습니다.")
                                sql="insert into t_macroinfo values(sysdate, '"+filename+"')"                                 
                                cur.execute(sql)
                            shutil.rmtree(resultdir)                        

os.putenv('NLS_LANG','KOREAN_KOREA.KO16MSWIN949')   # 한글 깨지는 것을 방지함
con1=cx_Oracle.connect("system/12345678")  # 오라클 연결, @127.0.0.1:1521/XE
cur=con1.cursor()

searchDir = "c:/temp"
resultdir="c:/temp/result"
searchMacro(searchDir)

con1.commit()
cur.close()
con1.close()

c:/temp/test.docm
c:/temp/test.docm 은 매크로를 포함하고 있습니다.
c:/temp/test2.docx
c:/temp/오목눈이/test.docm
c:/temp/오목눈이/test.docm 은 매크로를 포함하고 있습니다.


In [2]:
import os
import pandas as pd
import cx_Oracle

os.putenv('NLS_LANG','KOREAN_KOREA.KO16MSWIN949')   # 한글 깨지는 것을 방지함
con1=cx_Oracle.connect("system/12345678")  # 오라클 연결, @127.0.0.1:1521/XE
cur=con1.cursor()

sql = 'SELECT * FROM t_macroinfo'
sq = pd.read_sql(sql, con=con1)
print(sq)
writer = pd.ExcelWriter('c:/temp/macroresult.xlsx', engine='xlsxwriter')
sq.to_excel(writer, sheet_name='Sheet1')
writer.close()
cur.close()
con1.close()

                TODAY             FILENAME
0 2020-08-09 19:05:26  c:/temp/LimBest.doc
1 2020-08-09 19:05:26     c:/temp/문서1.docm
